In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb
from datetime import datetime
from dateutil.relativedelta import relativedelta
from functools import cached_property, partial
import plotly.express as px
import re

class DataManipulation:
    def _subset_by_dates(self, df, start_date=None, end_date=None):
        if start_date is None:
            start_date = datetime.today()-relativedelta(years=5)
        if end_date is None:
            end_date = datetime.today()
        return df[(df.index >= start_date)&(df.index <= end_date)]
    
    def _calculate_returns(self, df, start_date=None, end_date=None, log=False):
        df = self._subset_by_dates(df, start_date, end_date)
        if log is True:
            return np.log(df.pct_change()+1)
        else:
            return df.pct_change()
    
    def _get_market_data(self, start_date=None, end_date=None, market_ticker="^GSPC"):
        return wb.DataReader(market_ticker, data_source='yahoo',
                                             start='1970-1-1')['Close']
        

In [41]:
class Stock(DataManipulation):
    def __init__(self,ticker):
        self.symbol = ticker
        self.historical_data = pd.DataFrame()
        self.historical_data[ticker] = wb.DataReader(self.symbol, data_source='yahoo',
                                             start='1970-1-1')['Close']
    
    def volatility(self):
        # Assumes 5 year volatility
        five_years_ago = (datetime.today()-relativedelta(years=5)).strftime("%Y-%m-%d")
        tday = datetime.today().strftime("%Y-%m-%d")
        temp_df = self._subset_by_dates(self.historical_data, 
                                   five_years_ago, tday)
        temp_var = temp_df.values.std()
        return temp_var
    
    def plot(self, start_date=None, end_date=None):
        temp_df = self._subset_by_dates(self.historical_data, start_date, end_date)
        fig = px.line(temp_df, title=f"{self.symbol} Stock Price")
        return fig.show()
    
    def plot_returns(self, start_date=None, end_date=None):
        temp_df = self._calculate_returns(
                    self._subset_by_dates(
                        self.historical_data, start_date, end_date))
        return px.line(temp_df).show()
            
    def compare_to_market(self, start_date=None, end_date=None, market_ticker="^GSPC"):
        market_df = pd.DataFrame()
        market_df[market_ticker] = self._subset_by_dates(self._get_market_data(market_ticker),start_date,end_date)
        df = self._subset_by_dates(self.historical_data, start_date, end_date) 
        mx = df.join(market_df)
        temp_df = mx/mx.iloc[0]
        return px.line(temp_df, labels={'x':market_ticker}).show()
        
    class Position:
        
        

In [88]:
transactions_path = "https://s3.amazonaws.com/www.yourdatasurfer.com/test_stock_transactions.csv"
investment_transactions = pd.read_csv(transactions_path)
investment_transactions['Open date'] = pd.to_datetime(investment_transactions['Open date'])
investment_transactions.head()

,Symbol,Security,Qty,Type,Open date,Adj cost per share,Adj cost
0,AMZN,AMAZON COM INC,4,Buy,2014-01-06,390.87,1563.48
1,FB,FACEBOOK INC CL A,60,Buy,2014-01-06,54.79,3287.40
2,GLW,CORNING INC,100,Buy,2014-01-06,17.85,1784.99
3,GOOG,ALPHABET INC-CL C,2,Buy,2014-01-06,554.58,1109.16
4,GOOGL,ALPHABET INC-CL A,2,Buy,2014-01-06,557.89,1115.77


In [128]:
class Position(DataManipulation):
    def __init__(self, Stock):
        self._stock = Stock
        self._historical_stock_data = self._stock.historical_data
        self.shares_owned = 0
        self.transaction_history = pd.DataFrame()
        
    def buy(self, quantity, date, price=None):
#         self.transaction_history = 
        pass
    
    def update(self, date):
        pass
    
    def sell(self, quantity, date, price=None):
        if price is None:
            print(f"You do not own enough shares of {self._stock.symbol} to sell {quantity} shares on {date}.\nPosition will be updated.")
            return self.update(date)
        
    def load_transaction_history(self, path):
        filetype = re.findall("\.\w{2,4}$",path)[0]
        if filetype == ".csv":
            temp_df = pd.read_csv(path)
        elif filetype == [".xlsx",".xlsm",".xls"]:
            temp_df = pd.read_excel(path)
        else:
            print(f"LoadError: are you sure the path {path} is correct? Make sure your file is in the correct format.")
        temp_df = temp_df[temp_df['Symbol']==self._stock.symbol].reset_index(drop=True)
        temp_df['Open date'] = pd.to_datetime(temp_df['Open date'])
        self.transaction_history = temp_df
        
            

In [129]:
aapl = Stock("AAPL")

In [130]:
aapl.historical_data.head()

,AAPL
Date,
1980-12-12,0.128348
1980-12-15,0.121652
1980-12-16,0.112723
1980-12-17,0.115513
1980-12-18,0.118862


In [131]:
pos = Position(aapl)
pos.load_transaction_history(transactions_path)
pos.transaction_history

,Symbol,Security,Qty,Type,Open date,Adj cost per share,Adj cost
0,AAPL,APPLE INC,30,Buy,2015-08-26,106.79,3203.70
1,AAPL,APPLE INC,22,Buy,2017-09-27,154.66,3402.43
